In [4]:
import damask
import numpy as np
from pathlib import Path

# IPF Visualization for pre-processing

In [6]:
vti_path = 'reference_input/10grains_voronoi_5x5x5.vti'
material_path = 'reference_input/material_fullfield_10grains_Al.yaml'

v = damask.VTK.load(vti_path)
material_ID = v.get(label='material').flatten()

ma = damask.ConfigMaterial.load(material_path)

phases = list(ma['phase'].keys())
info = []

for m in ma['material']:
    c = m['constituents'][0]
    phase = c['phase']
    info.append({'phase':   phase,
                 'phaseID': phases.index(phase),
                 'lattice': ma['phase'][phase]['lattice'],
                 'O':       c['O'],
                })

l = np.array([0,0,1])                            # lab frame direction for IPF

IPF = np.ones((len(material_ID),3),np.uint8)
for i,data in enumerate(info):
    IPF[np.where(material_ID==i)] = \
    np.uint8(damask.Orientation(data['O'],lattice=data['lattice']).IPF_color(l)*255)

v = v.set(f'IPF_{l}',IPF)

p   = np.array([d['phase'] for d in info])
pid = np.array([d['phaseID'] for d in info])
v = v.set(label='phase',data=p[material_ID],info='phase name')
v = v.set(label='phaseID',data=pid[material_ID],info='phase ID')

v.save(f'IPF+phase_{Path(vti_path).stem}_{Path(material_path).stem}')